In [1]:
import pandas as pd
import numpy as np

In [60]:
all_games = pd.read_csv('nfl-football-player-stats/games_1995.csv',index_col='player_id')
all_players = pd.read_csv('nfl-football-player-stats/players_1995.csv',index_col='player_id')

In [35]:
half_ppr = {
    'rushing_yards': 0.1,
    'rushing_touchdowns': 6,
    'receiving_receptions': 0.5,
    'receiving_yards': 0.1,
    'receiving_touchdowns': 6
}

In [36]:
subset_position = {
    'RB': [['rushing_attempts', 'receiving_targets'], 5]
}

In [64]:
def get_playerids_thatweek(all_games, all_players, position, year, nfl_week, subset_position):
    ids = all_players[all_players.position == position].index
    games = all_games.loc[ids]
    stats = subset_position[position][0]
    sum_threshold = subset_position[position][1]
    worth_predicting = games[(games.year == year) & (games.game_number == nfl_week) &
                   (np.sum(games[stats], axis=1) >= sum_threshold)]
    ids = worth_predicting.index
    for_df = players.loc[ids, 'name']
    return pd.DataFrame(data=for_df)

In [153]:
a = get_playerids_thatweek(all_games, all_players,'RB',2011,5,subset_position)
a.shape

(57, 1)

In [276]:
1:3

SyntaxError: illegal target for annotation (<ipython-input-276-354ff4bd3bf9>, line 1)

In [333]:
def get_features_response(players, all_games, year, nfl_week, points_dict):
    games = all_games.loc[players.index]
    
    # compute fpts for each row
    games['fpts'] = np.zeros((len(games),1))
    for stat, value in zip(points_dict.keys(), points_dict.values()):
        games['fpts'] = games['fpts'] + games[stat]*value
    
    prev_years = games[(games.year < year) &
                       (games.game_number <= 16)]
    current_year = games[(games.year == year) &
                         (games.game_number < nfl_week)]
    rest_year = games[(games.year == year) &
                     (games.game_number>=nfl_week)]
    df = pd.concat((prev_years, current_year))
    df.sort_values(by=['player_id', 'year', 'game_number'], axis=0,
                   ascending=False, inplace=True)
    
    
        
    # group dataframe by index
    df = df.groupby(df.index)
    num_prev = df.fpts.agg('count').rename('num_prev',inplace=True)
    last = df.nth([0]).groupby('player_id').agg('mean').fpts.rename('last', inplace=True)
    next_3 = df.nth(list(range(1,4))).groupby('player_id').agg('mean').fpts.rename('next_3', inplace=True)
    next_15 = df.nth(list(range(4,19))).groupby('player_id').agg('mean').fpts.rename('next_15', inplace=True)
    to_debut = df.nth(list(range(19,300))).groupby('player_id').agg('mean').fpts.rename('to_debut', inplace=True)
    
    ros_ppg = rest_year.groupby('player_id').fpts.mean().rename('ros_ppg', inplace=True)

    for_return = players.join([num_prev,last, next_3, next_15, to_debut, ros_ppg], how='left')#,last,next_3,next_15, to_debut])
    return for_return
    
    
    

In [334]:
get_features_response(a, all_games, 2011, 5, half_ppr)

,name,num_prev,last,next_3,next_15,to_debut,ros_ppg
player_id,,,,,,,
22024,Daniel Thomas,2.0,19.7,12.200000,NaN,NaN,4.127273
10839,Steven Jackson,103.0,14.4,8.100000,14.066667,15.808333,14.708333
22426,Ryan Torain,14.0,19.5,7.100000,12.330000,NaN,1.685714
10524,Kendall Hunter,4.0,11.0,3.733333,NaN,NaN,5.128571
11580,Felix Jones,40.0,10.1,11.533333,10.240000,7.709524,7.175000
18312,Isaac Redman,21.0,5.7,4.600000,3.100000,0.950000,6.030769
22656,Michael Turner,106.0,21.8,12.866667,14.546667,7.032184,13.076923
8152,Frank Gore,88.0,20.9,9.133333,18.113333,14.449275,12.271429
13751,Marshawn Lynch,61.0,13.2,5.133333,8.926667,11.480952,18.636364


In [22]:
    if num_prev > 0:
        last = df['fpts'][0:1].item()
    if num_prev >= 4:
        next_3 = np.round(np.median(df['fpts'][1:4]),2)
    if num_prev >= 19:
        next_15 = np.round(np.median(df['fpts'][4:19]),2)
    if num_prev >= 20:
        to_debut = np.round(np.median(df['fpts'][19:]), 2)
    player = players[players.index == p_id]
    for_df.append([player.name.item(),player.position.item(),year,nfl_week,num_prev,last,next_3,next_15,to_debut])
    df = pd.DataFrame(for_df, index=player_ids, columns=['name','position','year','nfl_week','num_prev', 
                                                         'last', 'next_3','next_15','to_debut'])
    return df

def get_ros_stats(games, players, player_ids, year, nfl_week, points_dict):
    # Returns pandas Dataframe with columns player_id, name, year, nfl_week, ROS_ppg, ROS_games
    for_df = []
    for p_id in player_ids:  
        ROS = games[(games.index == p_id) & (games.year == year) & 
                    (games.game_number >= nfl_week) & (games.game_number <= 16)]
        ROS_games = len(ROS)
        ROS_pts = 0
        for stat, value in zip(points_dict.keys(), points_dict.values()):
            ROS_pts += sum(ROS[stat])*value
        if ROS_games > 0:
            ROS_ppg = np.round(ROS_pts/ROS_games,2)
        else:
            ROS_ppg = 0
        name = players[players.index == p_id]['name'].item()
        for_df.append([name, year, nfl_week, ROS_ppg, ROS_games])
    df = pd.DataFrame(data=for_df,index=player_ids, columns =['name', 'year', 'nfl_week', 'ROS_ppg', 'ROS_games'])
    return df

,name,year,nfl_week,ROS_ppg,ROS_games
17463,Adrian Peterson,2011,5,16.61,8
7173,Arian Foster,2011,5,23.05,11


In [18]:
ids = get_playerids_thatweek(games,players,'RB',2011,5,subset_position)
feats = get_features(games, players, ids, 2011, 5, half_ppr)
ros = get_ros_stats(games, players, ids, 2011, 5, half_ppr)
feat_list = ['num_prev', 'last', 'next_3', 'next_15', 'to_debut']
resp_list = ['ROS_ppg']

In [32]:
# get all features and responses ready
POSITION = 'RB'
YEAR = 2011
NFL_WEEK = 5

for year in [YEAR-1, YEAR]:   
    if year == YEAR:
        week_limit = NFL_WEEK-1
    else:
        week_limit = 16
    features_list = []
    responses_list = []
    for week in range(1,week_limit+1):
        ids = get_playerids_thatweek(games,players,POSITION,YEAR,NFL_WEEK,subset_position)
        feats = get_features(games, players, ids, year, week, half_ppr)
        ros_ppg = get_ros_stats(games, players, ids, year, week, half_ppr)
        features_list.append(feats)
        responses_list.append(ros_ppg)

features = pd.concat(features_list)
responses = pd.concat(responses_list)